# <font color="red">Warning: I don't have time over Christmas to comment this properly, or describe what I'm doing, so the following notebook shouldn't be taken as a good way of doing things!</font>

<font color="red">"First part of this looks pretty straightforward, so I'm a bit concerned about part 2."</font>

Looks like I was right to be concerned. Well I've got much better things to do with my Christmas than to debug an artificially bug-ridden piece of code, so I'm going to leave this until Boxing Day at the earliest.

Don't like OO programming, but this seems to lend itself to objects (python not being as amenable to closures as a proper functional language)

I'll tweak the definition of `State` so that the initial value of `a` is 1, and the remaining registers are 0. I also want method `registers`, `current_line` and `current_command` which return the current registers, the current line of the program, and the current command respectively.

In [1]:
class State:
    '''
    a class containing the program itself, current command
    being executed, and the registers
    '''
    def __init__(self, programIn_str, registersIn_dict={}):
        "programIn_str is the program as a string"
        
        self.pos=0
        self.registers_dict={r:0 for r in 'abcdefgh'}
        self.registers_dict.update(registersIn_dict)

        # parse the input string into a list of triples
        self.program_ls=[tuple(a.split()) for a in 
                         [nl for nl in programIn_str.strip().split('\n')]]
        
    # Now the methods. The program is halted if the
    # current position is outside the list of commands:
    def halted(self):
        "True if halted, False otherwise"
        return (self.pos<0) or (self.pos>=len(self.program_ls))
    
    # and execute the next command
    def execute_next_command(self):
        '''
        Execute command and move to the next state. Returns the
        command that was executed.
        '''
        (command, arg1, arg2)=self.program_ls[self.pos]
        if command=='set':
            if arg2 in self.registers_dict:
                self.registers_dict[arg1]=self.registers_dict[arg2]
            else:
                self.registers_dict[arg1]=int(arg2)
            self.pos+=1
            
        elif command=='sub':
            if arg2 in self.registers_dict:
                self.registers_dict[arg1] -= self.registers_dict[arg2]
            else:
                self.registers_dict[arg1] -= int(arg2)
            self.pos+=1
        
        elif command=='mul':
            if arg2 in self.registers_dict:
                self.registers_dict[arg1] *= self.registers_dict[arg2]
            else:
                self.registers_dict[arg1] *= int(arg2)
            self.pos+=1
        
        elif command=='jnz':
            if arg1 in self.registers_dict:
                x=self.registers_dict[arg1]
            else:
                x=int(arg1)
            if x==0:
                self.pos += 1
            else:
                if arg2 in self.registers_dict:
                    self.pos += self.registers_dict[arg2]
                else:
                    self.pos += int(arg2)
        

        return ((command, arg1, arg2))

    def registers(self, format="string"):
        '''
        Returns the current registers. If format="string", return
        as a string, if format="dict", return as a dict
        '''
        if format=="dict":
            return self.registers_dict
        elif format=="string":
            return ' '.join(['{}:{}'.format(r, v) for (r, v) in self.registers_dict.items()])
        else:
            raise ValueError('format not valid')

    def current_line(self):
        '''
        Returns the line of the program to be executed
        '''
        return self.pos

    def current_command(self):
        '''
        Returns the command of the program to be executed
        '''
        return self.program_ls[self.pos]


Now create a state with the puzzle input:

In [2]:
with open('data/day23.txt') as fIn:
    puzzleState=State(fIn.read(), {'a':1})

For this part of the task, want to cap the number of steps:

In [3]:
maxSteps_i=1000


with open('out.txt', 'w') as fOut:
    while puzzleState.registers_dict['d']<7:
        fOut.write(str(puzzleState.registers()))
        fOut.write('\n\n')
        fOut.write(str(puzzleState.current_command()))
        fOut.write('\n')
        com_tuple=puzzleState.execute_next_command()


In [4]:
progTest_str='''set b 19
set c 104
set f 1
set d 2
set e 2
set g d
mul g e
sub g b
jnz g 2
set f 0
sub e -1
set g e
sub g b
jnz g -8
sub d -1
set g d
sub g b
jnz g -13
jnz f 2
sub h -1
set g b
sub g c
jnz g 2
jnz 1 3
sub b -17
jnz 1 -23'''

progTest_str

'set b 19\nset c 104\nset f 1\nset d 2\nset e 2\nset g d\nmul g e\nsub g b\njnz g 2\nset f 0\nsub e -1\nset g e\nsub g b\njnz g -8\nsub d -1\nset g d\nsub g b\njnz g -13\njnz f 2\nsub h -1\nset g b\nsub g c\njnz g 2\njnz 1 3\nsub b -17\njnz 1 -23'

In [5]:
testState=State(progTest_str, {'a':1})

In [ ]:
maxLines_i=100000000

with open('out.txt', 'w') as fOut:
    while maxLines_i:
        fOut.write(str(testState.registers()))
        fOut.write('\n\n')
        fOut.write(str(testState.current_command()))
        fOut.write('\n')
        com_tuple=testState.execute_next_command()
        maxLines_i-=1


OK, a combination of examining the code and the output shows that what the program does is:

* From the starting values (b=79*100+100000=107900, c=b+17000=124900):
    * set d=e=2
    * set f=0
    * loop e from 2 to b, and then d from 2 to b
    * if b-(d*e) ever equals zero, then set f to 1, so that f=1 if b is non-prime
    * when d=e=b, if f==1, then increase h by 1
    * set f=0
    * increase b by 17
* When b==c, halt

The challenge is to find the value of h at the end of the run. So that will actually be the number of primes in the range (b, b+17, b+2*17, b+3*17, ..., c) inclusive.    

In [7]:
from math import sqrt

In [8]:
# Completely hacky way of doing primes.
# Yes, I do know how to do a Sieve of Eratosthenes, but it's
# not really necessary in this case...

def prime(num_i):
    "Return True if num_i is prime, False otherwise"
    return not any([(not num_i%i) for i in range(2, 1+int(sqrt(num_i)))])    

In [10]:
b=79
c=b
b*=100
b-= -100000
c=b
c-= -17000

# Use c+1 so that we include c

len([i for i in range(b, c+1, 17) if not prime(i)])

907

Is the right answer!